In [9]:
from dotenv import dotenv_values
from requests import get
from requests.models import HTTPError
from bs4 import BeautifulSoup

import pandas as pd
import random

In [18]:
CONFIG = dotenv_values('../.env')

In [6]:
def requisicao_api_alura():
    try:
        response = get('https://cursos.alura.com.br/api/categorias')
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'Um erro HTTP ocorreu: {http_err}')
    except Exception as err:
        print(f'Um erro ocorreu: {err}')
    else:
        return response.json()

In [8]:
dados = requisicao_api_alura()
urls_categorias = {}
url = 'https://cursos.alura.com.br/forum/categoria-'
for item in dados:
    urls_categorias[item['nome']] = f'{url}{item["slug"]}/todos'
urls_categorias

{'Programação': 'https://cursos.alura.com.br/forum/categoria-programacao/todos',
 'Front-end': 'https://cursos.alura.com.br/forum/categoria-front-end/todos',
 'Data Science': 'https://cursos.alura.com.br/forum/categoria-data-science/todos',
 'DevOps': 'https://cursos.alura.com.br/forum/categoria-devops/todos',
 'UX & Design': 'https://cursos.alura.com.br/forum/categoria-design-ux/todos',
 'Mobile': 'https://cursos.alura.com.br/forum/categoria-mobile/todos',
 'Inovação & Gestão': 'https://cursos.alura.com.br/forum/categoria-inovacao-gestao/todos'}

In [14]:
def requisicao(url):
    cookies = {'caelum.login.token':CONFIG['COOKIE']}
    try:
        response = get(url, cookies=cookies)
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'Um erro HTTP ocorreu: {http_err}')
    except Exception as err:
        print(f'Um erro ocorreu: {err}')
    else:
        soup = BeautifulSoup(response.text,'html.parser')
        return soup

In [20]:
paginas_categorias = {}
for chave, valor in urls_categorias.items():
    soup = requisicao(valor)
    ultima_pagina = int(soup.find('nav', {'class': 'busca-paginacao-links'}).get_text().split()[-1])
    paginas_categorias[chave] = random.sample(range(1, ultima_pagina+1), 250)
paginas_categorias

{'Programação': [2562,
  2761,
  3426,
  898,
  329,
  2780,
  2837,
  802,
  2093,
  3605,
  3723,
  2859,
  852,
  247,
  3041,
  2133,
  2125,
  680,
  804,
  2267,
  2414,
  3171,
  2432,
  3319,
  294,
  3117,
  1237,
  1282,
  2408,
  462,
  2554,
  2039,
  2479,
  3253,
  465,
  3393,
  2457,
  1326,
  668,
  419,
  223,
  10,
  2117,
  1323,
  1382,
  3395,
  1223,
  335,
  3264,
  2681,
  2906,
  3055,
  2290,
  3766,
  2721,
  1020,
  3843,
  1558,
  3372,
  256,
  2241,
  3778,
  1904,
  1884,
  193,
  3638,
  2814,
  2237,
  2051,
  1169,
  656,
  3184,
  2739,
  3564,
  3631,
  2698,
  600,
  2539,
  2059,
  1386,
  2735,
  886,
  3460,
  1304,
  1895,
  2067,
  1868,
  3497,
  3609,
  2809,
  1749,
  3838,
  586,
  2470,
  518,
  3265,
  1379,
  448,
  3344,
  131,
  2672,
  2370,
  2284,
  2718,
  3118,
  115,
  2987,
  3819,
  3493,
  2243,
  2883,
  2417,
  1236,
  2931,
  954,
  858,
  3448,
  3075,
  1645,
  2072,
  466,
  343,
  3148,
  1221,
  2706,
  1586,
  933,


In [47]:
def dataframe_links_categoria(categoria):
    titulos = []
    links = []
    for pagina in paginas_categorias[categoria]:
        soup  = requisicao(f'{urls_categorias[categoria]}/{pagina}')
        itens  = soup.find_all('a', {'class': 'forumList-item-subject-info-title-link'})
        for item in itens:
            titulos.append(item.text.strip().split('\n')[0])
            links.append(f'https://cursos.alura.com.br{item["href"]}')

    dados = pd.DataFrame({'titulo': titulos, 'links': links})
    dados.to_csv(f'../dados/{categoria}.csv')

In [46]:
dataframe_links_categoria('Data Science')

In [48]:
dataframe_links_categoria('Programação')

In [49]:
dataframe_links_categoria('Inovação & Gestão')

In [50]:
dataframe_links_categoria('Front-end')

In [57]:
dataframe_links_categoria('DevOps')

In [58]:
dataframe_links_categoria('UX & Design')

In [59]:
dataframe_links_categoria('Mobile')